In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
from datasets import load_dataset

dataset = load_dataset("knkarthick/dialogsum")

original_test_dataset = dataset["test"]

def load_model_and_tokenizer(model_path):
    # Load the fine-tuned model and tokenizer
    model = BartForConditionalGeneration.from_pretrained(model_path)
    tokenizer = BartTokenizer.from_pretrained(model_path)
    return model, tokenizer


def generate_summaries(model, tokenizer, original_test_dataset, num_examples=5, max_source_length=512):
    model.eval()  # Set the model to evaluation mode
    model.to("cuda" if torch.cuda.is_available() else "cpu")  # Move model to the appropriate device

    for i in range(num_examples):
        input_dialogue = original_test_dataset[i]["dialogue"]
        input_ids = tokenizer.encode(
            input_dialogue, return_tensors="pt", max_length=max_source_length, truncation=True
        ).to(model.device)

        # Generate summary
        output_ids = model.generate(input_ids=input_ids, max_length=128, num_beams=4)
        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Print input and outputs
        print(f"Input Dialogue: {input_dialogue}")
        print(f"Generated Summary: {generated_summary}")
        print(f"Reference Summary: {original_test_dataset[i]['summary']}")
        print("-" * 50)


# Example Usage
model_path = "./bart-large-cnn-finetuned-dialogsum"  # Path to your saved model
model, tokenizer = load_model_and_tokenizer(model_path)
generate_summaries(model, tokenizer, original_test_dataset)

c:\Users\anime\anaconda3\Lib\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
c:\Users\anime\anaconda3\Lib\site-packages\transformers\generation\utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Input Dialogue: #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue w

In [ ]:

# def evaluate_model(decoded_preds, test_dataset):  # No need for model_name
#     rouge = evaluate.load("rouge")
#     references = test_dataset["summary"]
#     rouge_results = rouge.compute(predictions=decoded_preds, references=references)

#     print("ROUGE-1:", rouge_results["rouge1"])
#     print("ROUGE-2:", rouge_results["rouge2"])
#     print("ROUGE-L:", rouge_results["rougeL"])

#     P, R, F1 = score(decoded_preds, references, lang="en", verbose=True)
#     print("BERTScore F1:", F1.mean().item())
